In [1]:
%load_ext rpy2.ipython

In [2]:
%%R 
options(digits=5, max.print=100)  # Adjust numbers as needed
set.seed(123)

# library(Pando)
library(Seurat)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Signac)
library(EnsDb.Hsapiens.v86)

library(Matrix)
#-------- import atac-seq count matrix and metadata and creat assay
X <- readMM("../output/scATAC/matrix_annot/X_matrix.mtx")
X <- t(X)
annotation_peak <- read.csv("../output/scATAC/matrix_annot/annotation_peak.csv", row.names = 1)
annotation_cells <- read.csv("../output/scATAC/matrix_annot/annotation_cells.csv", row.names = 1)

# Filter out entries where seqname is not chr 
filter_indices <- grepl("^chr", annotation_peak$seqname)
annotation_peak_filtered <- annotation_peak[filter_indices, ]

# Filter the rows in X
X_filtered <- X[filter_indices, ]
  
peaks_matrix <- X_filtered
colnames(peaks_matrix) <- annotation_cells$obs_id 

rownames(peaks_matrix) = paste(annotation_peak_filtered$seqname, annotation_peak_filtered$ranges, sep = "_")
atac_assay <- CreateChromatinAssay(counts = peaks_matrix, ranges=GRanges(annotation_peak_filtered$seqname,
                              IRanges(annotation_peak_filtered$ranges)), 
                             colData = DataFrame(annotation_cells), genome = "hg38")
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevelsStyle(annotations) <- "UCSC"
genome(annotations) <- "hg38"
Annotation(atac_assay) <- annotations
#-------- import rna-seq count matrix (with row and col names already assigned ) and create seurat object
rna_short  = readRDS('../output/figr/rna_short.rds')
seurat_object <- CreateSeuratObject(count = rna_short, project = "pbmc", min.cells = 3, min.features = 200)
# add peaks
seurat_object[["peaks"]] = atac_assay

seurat_object


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
     |                                                  | 0 % ~calculating   |++                                                | 4 % ~49s           |++++                                              | 8 % ~43s           |++++++                                            | 12% ~42s           |++++++++                                          | 16% ~35s           |++++++++++                                        | 20% ~32s           |++++++++++++                                      | 24% ~29s           |++++++++++++++                                    | 28% ~27s           |++++++++++++++++                                  | 32% ~25s           |++++++++++++++++++                                | 36% ~23s           |++++++++++++++++++++                              | 40% ~21s           |++++++++++++

Loading required package: SeuratObject
Loading required package: sp
‘SeuratObject’ was built under R 4.1.2 but the current version is
4.3.2; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed
‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed

Attaching package: ‘SeuratObject’

The following object is masked from ‘package:base’:

    intersect

Loading required package: GenomeInfoDb
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following object is masked from ‘package:SeuratObject’:

    intersect

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, i